In [1]:
import cv2
import sys
import imutils
from imutils.video import FPS
import time

In [2]:
cap = cv2.VideoCapture('/Users/aaronestes/Downloads/jae16.MOV')

In [3]:
tracker_types = ['BOOSTING', 'MIL','KCF', 'TLD', 'MEDIANFLOW', 'CSRT', 'MOSSE', 'GOTURN']

In [4]:
tracker = cv2.TrackerTLD_create()

In [5]:
ok, frame = cap.read()

In [6]:
initBB = None

In [7]:
fps = None

In [8]:
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
fgbg = cv2.createBackgroundSubtractorMOG2()

In [9]:
if not cap.isOpened():
    print("Could not open video")
    sys.exit()

In [10]:
OPENCV_OBJECT_TRACKERS = {
    ## CSRT: high accuracy but low frames/second
    "csrt": cv2.TrackerCSRT_create,
    ## KCF: not very good with changes in z axis. does not take risks.
    "kcf": cv2.TrackerKCF_create,
    ## Boosting: the OG algorithm. Displayed the most correct matches for face!
    "boosting": cv2.TrackerBoosting_create,
    ## MIL follows putting green ball.
    "mil": cv2.TrackerMIL_create,
    ## TLD pros: best for club head. Takes the most attempts. The crackhead algorithm.
    "tld": cv2.TrackerTLD_create,
    ## Median Flow: works best when the motion is negligible.
    "medianflow": cv2.TrackerMedianFlow_create,
    ## MOSSE is smooth. Amazing for the putting green ball. increases and decreases in size the best.
    "mosse": cv2.TrackerMOSSE_create,
    ## GOTURN: only downside is objects that blend in to their background.
    "goturn": cv2.TrackerGOTURN_create
}

In [11]:
while True:
    ok, frame = cap.read()
    if frame is None:
        break
    frame = imutils.resize(frame, width=500)
    fgMask = fgbg.apply(frame)
    fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_OPEN, kernel)
    (H, W) = frame.shape[:2]
    if initBB is not None:
        (ok, bbox) = tracker.update(frame)
        if ok:
            (x,y,w,h) = [int(v) for v in bbox]
            cv2.rectangle(frame, (x,y), (x + w, y + h), (0,255,0), 2)
        fps.update()
        fps.stop()
    
    grayFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    dst = cv2.GaussianBlur(grayFrame,(5,5),cv2.BORDER_DEFAULT)
    ## assist the algorithm in processing
    cv2.imshow("frame", dst)
    
    
    k = cv2.waitKey(1) & 0xff
    if k == ord("s"):
        initBB = cv2.selectROI("frame", dst, fromCenter=False, showCrosshair=True)
        tracker.init(frame, initBB)
        fps = FPS().start()
        while True:
            k2 = cv2.waitKey(1) or 0xff
            cv2.imshow('frame', dst)
            if k2 == ord('s'):
                break
    elif k == ord("q"):
        break

cap.release()
# Closes all the frames 
cv2.destroyAllWindows()